In [5]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelBinarizer
import pickle

In [15]:
file = open('data_all.csv', 'rb')
data = pickle.load(file)
x_data,y_data = data
file.close()
x_data = np.concatenate((x_data[0], x_data[1],x_data[2]),axis=0)
y_data = np.concatenate((y_data[0], y_data[1],y_data[2]),axis= 0)

In [18]:
x_test = []
y_test = []
x_train = []
y_train = []
encoder = LabelBinarizer()
y_data =encoder.fit_transform(y_data)
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=0.3)

In [19]:
x_train.shape

(46505, 64, 64, 1)

In [20]:
def show_history():

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(accuracy, label='Training Accuracy')
    plt.plot(val_accuracy, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)  
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.ylim(0, 1)
    plt.legend()

    plt.show()

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard


# Khởi tạo mô hình LSTM 2D
input_shape = x_test[1].shape

model = Sequential()

# Tạo lớp Conv2D đầu tiên để chuyển hình ảnh xám thành 3 kênh (nhưng vẫn giữ nguyên kích thước 64x64)
# model.add(Conv2D(3, (3, 3), activation='relu', input_shape=input_shape))

# Lớp Convolution và Batch Normalization (BN) - Block 1
model.add(Conv2D(32, (3, 3), activation='relu', padding='same',input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2 ,2)))

# Lớp Convolution và Batch Normalization (BN) - Block 2
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2 ,2)))

# Lớp Convolution và Batch Normalization (BN) - Block 3
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2 ,2)))

# Lớp Convolution và Batch Normalization (BN) - Block 4
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2 ,2)))

# Lớp Convolution và Batch Normalization (BN) - Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# Lớp Fully Connected
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Thay num_classes bằng số lớp phân loại thực tế

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
model.summary()


2023-11-08 09:18:12.674568: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 09:18:12.674599: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 09:18:12.674632: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-08 09:18:12.682054: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-08 09:18:14.096742: I tensorflow/compiler/

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 64, 64, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 64, 64, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 32)        0         
 D)                                                              
                                                        

In [22]:
history = model.fit(x_train,y_train,batch_size=64, epochs=50,validation_data=(x_test, y_test))
show_history()

2023-11-08 09:18:21.466880: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 761937920 exceeds 10% of free system memory.
2023-11-08 09:18:22.111883: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 761937920 exceeds 10% of free system memory.


Epoch 1/50


2023-11-08 09:18:25.530642: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-08 09:18:26.942702: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 764.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-08 09:18:27.038792: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.34GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-08 09:18:27.131388: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3a7cdd0380 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-08 09:18:27.131411: I tensorflow/compiler/xla/service/service.cc:176]   S

726/727 [============================>.] - ETA: 0s - loss: 0.9741 - accuracy: 0.5415

2023-11-08 09:19:23.093331: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.25GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-08 09:19:23.228851: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.25GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


727/727 [==============================] - ETA: 0s - loss: 0.9739 - accuracy: 0.5416

2023-11-08 09:19:24.412993: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 326565888 exceeds 10% of free system memory.
2023-11-08 09:19:24.677468: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 326565888 exceeds 10% of free system memory.


727/727 [==============================] - 69s 82ms/step - loss: 0.9739 - accuracy: 0.5416 - val_loss: 1.4910 - val_accuracy: 0.5160
Epoch 2/50
  2/727 [..............................] - ETA: 51s - loss: 0.7805 - accuracy: 0.6328

2023-11-08 09:19:31.399921: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.19GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


727/727 [==============================] - 58s 80ms/step - loss: 0.8155 - accuracy: 0.6048 - val_loss: 1.2719 - val_accuracy: 0.4852
Epoch 3/50
727/727 [==============================] - 56s 77ms/step - loss: 0.7886 - accuracy: 0.6222 - val_loss: 1.1160 - val_accuracy: 0.5152
Epoch 4/50
727/727 [==============================] - 56s 77ms/step - loss: 0.7715 - accuracy: 0.6312 - val_loss: 1.0366 - val_accuracy: 0.6076
Epoch 5/50
727/727 [==============================] - 56s 77ms/step - loss: 0.7523 - accuracy: 0.6429 - val_loss: 0.9611 - val_accuracy: 0.5573
Epoch 6/50
727/727 [==============================] - 56s 77ms/step - loss: 0.7464 - accuracy: 0.6459 - val_loss: 1.0747 - val_accuracy: 0.5559
Epoch 7/50
727/727 [==============================] - 61s 83ms/step - loss: 0.7316 - accuracy: 0.6510 - val_loss: 0.9875 - val_accuracy: 0.6103
Epoch 8/50
727/727 [==============================] - 59s 82ms/step - loss: 0.7285 - accuracy: 0.6544 - val_loss: 0.9907 - val_accuracy: 0.5819
Epo